In [1]:
# from PIL import Image
import cv2
import dlib
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import math
# %matplotlib inline

In [2]:
def grid_display(list_of_images, list_of_titles=[], no_of_columns=2, figsize=(30,30), ratio=1, conv_color = False):
    
    if conv_color:
        list_of_images = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in list_of_images]
    
    num_img = len(list_of_images)
    row = math.ceil( num_img / no_of_columns )
    
    if figsize[1] == 0:
        fig = plt.figure(figsize=(figsize[0], figsize[0]*row/no_of_columns/ratio))
    else:
        fig = plt.figure(figsize=figsize)
        
    for i, img in enumerate(list_of_images):
        if list_of_titles:
            fig.add_subplot( row, no_of_columns , i+1, title = list_of_titles[i])
        else:
            fig.add_subplot( row, no_of_columns , i+1)
        plt.imshow(list_of_images[i])
        plt.axis('off')
    plt.show()

In [3]:
def get_lankmark(path):
    detector = dlib.get_frontal_face_detector()
    landmark_predictor = dlib.shape_predictor(
        r'models/shape_predictor_68_face_landmarks.dat')
    img = cv2.imread(path)
    faces = detector(img, 1)
    landmark = []
    rect = []
    if (len(faces) > 0):
        for k, d in enumerate(faces):
            rect.append( [( d.left(), d.top() ), ( d.right(), d.bottom() )] )
            
            shape = landmark_predictor(img, d)
            feas = []  #lankmark points
            for i in range(68):
                num = str(shape.part(i))[1:-1].split(",")
                feas.append( (int(num[0]), int(num[1])) ) 
            landmark.append(feas)
    
    return img, rect, landmark

In [4]:
def init_dir(organ_dirs, remove_old = True):
    for organ_dir in organ_dirs:
        if os.path.exists(organ_dir) and remove_old:
            shutil.rmtree(organ_dir)
        if not os.path.exists(organ_dir):
            os.mkdir(organ_dir)    

In [5]:
organ_dirs = {'jaw':'./jaw/', 
              'cheek_l':'./cheek_l/', 
              'cheek_r':'./cheek_r/', 
              'nose':'./nose/', 
              'lip':'./lip/', 
              'forehead':'./forehead/', 
              'neck':'./neck/'
             }

def get_organ_boxes(landmark):
    #define boxes coordinate
    return {
    #forehead
    'forehead':(
        landmark[0][19][0], #x_left_top
        landmark[0][19][1] - (landmark[0][23][0] - landmark[0][20][0]),  #y_left_top
        landmark[0][24][0], #x_right_bottom
        min(landmark[0][17][1], landmark[0][18][1], landmark[0][19][1], landmark[0][20][1]), #y_right_bottom
    ),
    #left cheek
    'cheek_l':(
        min(landmark[0][1][0], landmark[0][2][0], landmark[0][3][0]), #x_left_top
        max(landmark[0][36][1], landmark[0][39][1], landmark[0][40][1], landmark[0][41][1]), #y_left_top
        (landmark[0][31][0] + landmark[0][31][0] - landmark[0][32][0]), #x_right_bottom
        landmark[0][49][1], #y_right_bottom
    ),
    #right cheeka
    'cheek_r':(
        (landmark[0][35][0]+landmark[0][35][0]-landmark[0][34][0]), #x_left_top
        max(landmark[0][46][1], landmark[0][47][1], landmark[0][45][1], landmark[0][42][1]), #y_left_top
        landmark[0][15][0], #x_right_bottom
        landmark[0][63][1], #y_right_bottom
    ),
    #nose
    'nose':(
        (landmark[0][31][0] + landmark[0][31][0] - landmark[0][32][0]), #x_left_top
        landmark[0][27][1], #y_left_top
        (landmark[0][35][0]+landmark[0][35][0]-landmark[0][34][0]), #x_right_bottom
        max(landmark[0][31][1], landmark[0][32][1], landmark[0][33][1], landmark[0][34][1], landmark[0][35][1]), #y_right_bottom
    ),
    #lip
    'lip':(
        landmark[0][48][0], #x_left_top
        min(landmark[0][50][1], landmark[0][52][1]),  #y_left_top
        landmark[0][54][0], #x_right_bottom
        max(landmark[0][56][1], landmark[0][57][1], landmark[0][58][1]), #y_right_bottom
    ),
    #jaw
    'jaw': (
        landmark[0][5][0], #x_left_top
        landmark[0][57][1], #y_left_top
        landmark[0][11][0], #x_right_bottom
        landmark[0][8][1] #y_right_bottom
    ),
    'neck':(
        landmark[0][5][0], #x_left_top
        landmark[0][8][1], #y_left_top
        landmark[0][11][0], #x_right_bottom
        landmark[0][8][1] + landmark[0][8][1] - landmark[0][57][1], #y_right_bottom
    ),
    }


# init_dir(organ_dirs, remove_old = True)
idx = 0

In [21]:
def mask_lip(img, landmark):
    # extract lip shape
    # get countour from landmark
    countour = [np.array(landmark[0][48:60])] 
    # mask defaulting to black for 3-channel and transparent for 4-channel
    # (of course replace corners with yours)
    mask = np.zeros(img.shape, dtype=np.uint8)
    # fill the ROI so it doesn't get wiped out when the mask is applied
    channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
    ignore_mask_color = (255,)*channel_count
    cv2.fillPoly(mask, countour, ignore_mask_color)
    # from Masterfool: use cv2.fillConvexPoly if you know it's convex
    # apply the mask
    img_masked_lip = cv2.bitwise_and(img, mask)
    return img_masked_lip

In [ ]:
idx = idx+1
sample_dir = '../sample_sets/photo/'
# sample_dir = './test/'
imgs = []
face_imgs = os.listdir(sample_dir)
print (face_imgs)

for face_img in face_imgs[idx:idx+2]:
# for face_img in face_imgs:
    #extract landmark features
    img, rect, landmark = get_lankmark(sample_dir + face_img)
    
    organ_boxes = get_organ_boxes(landmark)

    

#     # cut different parts of face
#     for organ in organ_dirs:
# #         print (organ)
#         if organ == 'lip':
# #             img2 = mask_lip(img, landmark)
#             img2 = img
#             organ_crop = img2[ organ_boxes[organ][1]:organ_boxes[organ][3],
#                                                      organ_boxes[organ][0]:organ_boxes[organ][2] ]
# #             organ_crop = cv2.resize(organ_crop, (120,60))
#         else:
#             organ_crop = img[ organ_boxes[organ][1]:organ_boxes[organ][3],
#                                                      organ_boxes[organ][0]:organ_boxes[organ][2] ]
#         cv2.imwrite(organ_dirs[organ] + face_img, organ_crop)
   


    # draw face rect and landmark features
    for i, feat_point in enumerate(landmark[0]):
        cv2.circle(img, feat_point, 2, (0, 0, 0))
        cv2.putText(img, str(i), feat_point, 1, 0.7, (0, 0, 0))
    for organ in organ_dirs:
        cv2.rectangle(img, (organ_boxes[organ][0], organ_boxes[organ][1]), (organ_boxes[organ][2],organ_boxes[organ][3]),
                      (255, 255, 255), 2)
    cv2.rectangle(img, rect[0][0], rect[0][1], (0, 0, 255), 2)

    # resize image
#     img_s = cv2.resize(img, (0,0), fx=0.5, fy=0.5)
    imgs.append(img)
    
images = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in imgs]
grid_display(images, [], 1, (40,30), 1, False)
